In [ ]:
#javascript to get img urls (run in browser console after google img search for 'teddy bear'. Note: disable add blockers to get the csv file in separate window)
#urls=Array.from(document.querySelectorAll('.rg_i')).map(el=> el.hasAttribute('data-src')?el.getAttribute('data-src'):el.getAttribute('data-iurl'));
#window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *
from fastai.widgets import *
#from google.colab import drive
#drive.mount('/content/drive')

image_type='patents'
learn_pkl_name = image_type + '_image_classifer_resnet50_export.pkl'
#path = Path('/content/drive/My Drive/data/' + image_type)
path = Path('/home/jupyter/data/'+image_type)
classes = os.listdir(path)
for c in classes:
  folder = c
  #file = 'urls_'+c+'.csv'
  dest = path/folder
  if not os._exists(dest):
    dest.mkdir(parents=True, exist_ok=True)
    #download_images(path/file, dest, max_pics=10)
for c in classes: 
  print(c)
  verify_images(path/c, delete=True, max_workers=8)
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)
data.classes
data.show_batch(rows=3, figsize=(7,8))
data.classes, data.c, len(data.train_ds), len(data.valid_ds)
#Train
learn = cnn_learner(data, models.resnet50, metrics=error_rate)
learn.lr_find()
learn.recorder.plot()
learn.fit_one_cycle(1) #todo original 8
learn.save('stage-1-50')
learn.unfreeze()
learn.fit_one_cycle(1, max_lr=slice(1e-6, 1e-4)) #todo epochs should be 3
learn.load('stage-1-50');
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
learn.export(path/learn_pkl_name)
#defaults.device = torch.device('cpu')
#img = open_image(path/'advertising'/'1.BANGADSMEDIA_51.jpg')
#img
learn = load_learner(path, learn_pkl_name)
pred_class,pred_idx,outputs=learn.predict(img)
pred_class


In [ ]:
#img = open_image(path/'carpets'/'1.fd7b9276073ab01dde5a6ed97e1ba95e.jpg')
#img = open_image(path/'compositions_ceramic'/'5.bottle-support-patent-drawing-from-1937-mugs.jpg')
#img
learn = load_learner(path,learn_pkl_name)
pred_class,pred_idx,outputs=learn.predict(img)
pred_class
